## Description:
这个jupyter主要是处理模板图像， 读取，然后计算轮廓图，把模板图像划分成一个个的小格子

In [1]:
import numpy as np
import cv2
from imutils import contours
import pickle

In [2]:
def cv_show(title, img):
    cv2.imshow(title, img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [3]:
# 读取模板图像
img = cv2.imread("images/ocr_a_reference.png")   # 读取的时候转灰度 cv2.imread("images/ocr_a_reference.png", 0)
# 转成灰度图
template = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
# 二值图像
template = cv2.threshold(template, 10, 255, cv2.THRESH_BINARY_INV)[1]
# cv_show('template', template)

In [4]:
cv_show('template', template)

In [21]:
template.shape

(126, 800)

## 计算轮廓
cv2.findContours()函数接受的参数为二值图， 即黑白图像(不是灰度图), cv2.RETR_EXTERNAL只检测外轮廓， cv2.CHAIN_APPROX_SIMPLE只保留终点坐标

In [7]:
# 最新版opencv只返回两个值了 3.2之后， 不会返回原来的二值图像了，直接返回轮廓信息和层级信息
contourss, hierarchy = cv2.findContours(template.copy(), cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)

In [14]:
len(contourss)  # 10个轮廓

10

In [8]:
# 可视化一下
cv2.drawContours(img,contourss,-1,(0,0,255),3) 
# cv_show('template', img)

array([[[255, 255, 255],
        [255, 255, 255],
        [255, 255, 255],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 255, 255]],

       [[255, 255, 255],
        [255, 255, 255],
        [255, 255, 255],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 255, 255]],

       [[255, 255, 255],
        [255, 255, 255],
        [255, 255, 255],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 255, 255]],

       ...,

       [[255, 255, 255],
        [255, 255, 255],
        [255, 255, 255],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 255, 255]],

       [[255, 255, 255],
        [255, 255, 255],
        [255, 255, 255],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 255, 255]],

       [[255, 255, 255],
        [255, 255, 255],
        [255, 255, 255],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 255, 255]]

In [10]:
cv_show('template', img)

In [17]:
# 下面将轮廓进行排序，这是因为必须保证轮廓的顺序是0-9的顺序排列着
def sort_contours(cnts, method='left-to-right'):
    reverse = False
    i = 0
    if method == 'right-to-left' or method == 'bottom-to-top':
        reverse = True
    if method == 'top-to-bottom' or method == 'bottom-to-top':
        i = 1
    
    boundingBoxes = [cv2.boundingRect(c) for c in cnts]  # 用一个最小矩形，把找到的形状包起来x,y,h,w
    
    # 根据每个轮廓左上角的点进行排序， 这样能保证轮廓的顺序就是0-9的数字排列顺序
    (cnts, boundingBoxes) = zip(*sorted(zip(cnts, boundingBoxes), key=lambda x:x[1][i], reverse=reverse))
    
    return cnts, boundingBoxes 

In [18]:
refCnts = sort_contours(contourss, method='left-to-right')[0]  

In [26]:
# 每个轮廓进行数字编号
digits2Cnt = {}
# 遍历每个轮廓
for i, c in enumerate(refCnts):
    # 计算外接矩形，并且resize成合适大小
    (x, y, w, h) = cv2.boundingRect(c)
    # 单独把每个数字框拿出来 坐标系竖着的是y， 横着的是x
    roi = template[y:y+h, x:x+w] 
    # 重新改变大小
    roi = cv2.resize(roi, (57, 88))
    
    # 框与字典对应
    digits2Cnt[i] = roi

In [29]:
# 把处理好的模板进行保存
pickle.dump(digits2Cnt, open('digits2Cnt.pkl', 'wb'))